Decision Tree.
First of all we will load libraries needed

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler
import eli5

Loading data

In [2]:
train =  pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sampleSubmission = pd.read_csv("sampleSubmission.csv")


Removing variables we want to predict from train file

In [3]:
set(train.columns) - set(test.columns)
train.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


Removing columns from sampleSubmission

In [4]:
set(sampleSubmission.columns) - set(test.columns)
sampleSubmission.shape

(6493, 2)

In [5]:
sampleSubmission.head()

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


Now change date format

In [6]:
train["datetime"] = pd.to_datetime(train["datetime"])
train["datetime"].describe()

count                   10886
unique                  10886
top       2011-06-09 04:00:00
freq                        1
first     2011-01-01 00:00:00
last      2012-12-19 23:00:00
Name: datetime, dtype: object

In [7]:
def transform_dates(dataframe):
    train["dayofweek"] = train["datetime"].dt.dayofweek      #The day of the week with Monday=0, Sunday=6
    train["year"] = train["datetime"].dt.year
    train["month"] = train["datetime"].dt.month
    train["day"] = train["datetime"].dt.day
    train["hour"] = train["datetime"].dt.hour
    train["minute"] = train["datetime"].dt.minute
    train["second"] = train["datetime"].dt.second
    return dataframe

In [8]:
train = transform_dates(train)
train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,dayofweek,year,month,day,hour,minute,second
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,5,2011,1,1,0,0,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,5,2011,1,1,1,0,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,5,2011,1,1,2,0,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,5,2011,1,1,3,0,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,5,2011,1,1,4,0,0
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1,5,2011,1,1,5,0,0
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2,5,2011,1,1,6,0,0
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3,5,2011,1,1,7,0,0
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8,5,2011,1,1,8,0,0
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,6,14,5,2011,1,1,9,0,0


Rozpoczynam tworzenie drzewa

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score # ogólna funkcja do CV
from sklearn.model_selection import StratifiedKFold # podział na zbiory do CV


In [10]:
X = train.drop(["count","registered","casual","datetime"], axis = 1)
y = train["count"]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.33, random_state=42)

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [13]:
best_score = -np.inf
parameters = {}

In [14]:
for depth in range (1,11):
    for min_split in range(2,21):
        for min_leaf in range(1,21):
            clf = DecisionTreeRegressor(max_depth = depth, min_samples_split = min_split, 
                                             min_samples_leaf = min_leaf)
            scores = cross_val_score(clf, X, y, cv = 5,scoring='neg_mean_squared_error')
            if scores.mean() > best_score:
                #print("New best parameters!")
                parameters["best_depth"] = depth
                parameters["best_min_split"] = min_split
                parameters["best_min_leaf"] = min_leaf
                best_score = scores.mean()
                print("Accuracy: {}".format(scores.mean()))
                print("max_depth: {}, min_samples_split: {}, min_samples_leaf: {}".format(depth, 
                                                                                         min_split, 
                                                                                         min_leaf, 
                                                                                         ))

Accuracy: -25682.901809311556
max_depth: 1, min_samples_split: 2, min_samples_leaf: 1
Accuracy: -24894.989827302557
max_depth: 3, min_samples_split: 2, min_samples_leaf: 1
Accuracy: -19784.754570820358
max_depth: 4, min_samples_split: 2, min_samples_leaf: 1
Accuracy: -16734.903643373567
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1
Accuracy: -16682.005246541597
max_depth: 5, min_samples_split: 2, min_samples_leaf: 3
Accuracy: -16681.987658953323
max_depth: 5, min_samples_split: 2, min_samples_leaf: 4
Accuracy: -16681.618328993027
max_depth: 5, min_samples_split: 2, min_samples_leaf: 6
Accuracy: -16681.591052312087
max_depth: 5, min_samples_split: 2, min_samples_leaf: 11
Accuracy: -16675.377808838733
max_depth: 5, min_samples_split: 2, min_samples_leaf: 16
Accuracy: -16675.003540812453
max_depth: 5, min_samples_split: 2, min_samples_leaf: 17
Accuracy: -16674.95718057999
max_depth: 5, min_samples_split: 8, min_samples_leaf: 16
Accuracy: -15552.791922182287
max_depth: 6, min_sam

In [15]:
regressor = DecisionTreeRegressor(max_depth = parameters["best_depth"],
                                  min_samples_split = parameters["best_min_split"],
                                  min_samples_leaf = parameters["best_min_leaf"])

In [16]:
#from sklearn.model_selection import GridSearchCV
#parameters = {'max_depth':range(14,16),'min_samples_split':range(2,21),'min_samples_leaf':range(1,21)}
#tree = DecisionTreeRegressor()
#klasyfikator = GridSearchCV(estimator = tree, param_grid = parameters, return_train_score=True, verbose=0,scoring='neg_mean_squared_error')
#klasyfikator.fit(X = X_train, y = y_train)
#klasyfikator.best_score_
#klasyfikator.best_params_

In [17]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y))**2))

rmsle_score = make_scorer(rmsle, greater_is_better = False)

In [18]:
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=4,
           min_samples_split=13, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [19]:
X_test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,year,month,day,hour,minute,second
3133,3,0,1,1,33.62,40.150,59,0.0000,1,2011,7,19,11,0,0
5786,1,1,0,1,4.10,6.820,54,6.0032,0,2012,1,16,6,0,0
5224,4,0,0,1,9.84,11.365,48,12.9980,6,2011,12,11,18,0,0
8953,3,0,1,2,29.52,34.090,62,12.9980,2,2012,8,15,10,0,0
8054,2,0,1,1,25.42,31.060,53,16.9979,4,2012,6,15,23,0,0


In [20]:
X_train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,year,month,day,hour,minute,second
1101,1,0,1,3,17.22,21.210,0,15.0013,3,2011,3,10,12,0,0
4194,4,0,1,1,21.32,25.000,59,7.0015,3,2011,10,6,19,0,0
10389,4,0,0,1,11.48,13.635,81,12.9980,6,2012,11,18,7,0,0
6969,2,0,0,1,25.42,30.305,22,19.9995,6,2012,4,8,17,0,0
313,1,0,1,1,9.02,9.850,37,26.0027,4,2011,1,14,13,0,0


In [21]:
predicted = regressor.predict(X_test)

In [22]:
eli5.explain_weights(regressor, feature_names=X_train.columns.values)

Weight,Feature
0.6006,hour
0.1171,workingday
0.0959,year
0.0456,month
0.0440,season
0.0367,atemp
0.0171,temp
0.0156,dayofweek
0.0137,humidity
0.0116,weather


In [23]:
sampleSubmission = transform_dates(sampleSubmission)
sampleSubmission = sampleSubmission.drop(["datetime"], axis = 1)

In [24]:
predicted = regressor.predict(X_test) #robimy predyckję
wyniki = pd.DataFrame(y_test) 
wyniki["y_pred"] = predicted #dokljamy kolumnę z predykcją
wyniki.rename(columns = {"Klasa":"y_test"}, inplace = True)

In [25]:
wyniki

,count,y_pred
3133,127,153.557971
5786,13,2.833333
5224,163,233.391304
8953,233,208.248227
8054,222,213.600000
10044,166,202.444444
5337,144,166.547619
2753,376,380.396040
10127,601,701.577778
33,53,64.250000


In [26]:
wewnetrzny_test  = rmsle(y_pred = wyniki["y_pred"], y = wyniki["count"])
print(wewnetrzny_test)

0.40937801734208534
